In [15]:
import yfinance as yf 
import pandas as pd
import numpy as np
from datetime import datetime

import quandl

import sys
import urllib
import os
from matplotlib import pylab as plt 

import quandl

import seaborn as sns

import sys

from scipy.stats import norm
import ezodf

## Functions 

In [16]:
def import_MSCI(name):
    '''Function to clean data from MSCI
    https://www.msci.com/end-of-day-data-country
    '''
    
    path = "../Data/" + name + ".xls"
    imp_tmp = pd.read_excel(path)
    price_type = imp_tmp.loc[1][1]

    imp = imp_tmp[6:]
    imp.columns = ['Date', name]
    imp = imp.dropna()
    imp = imp.set_index('Date')
    imp.index = imp.index.strftime('%Y/%m')
    imp.index = pd.to_datetime(imp.index, errors='coerce') # sets to first day of month 
    imp.index = imp.index.shift(1, freq='MS') # raw data is end of month, set to first day next month
    
    return imp

In [17]:
def wrangling(securities, assets_owned = 1):
    
    data = pd.concat(securities, axis = 1)
    data = data.dropna()
    
    mom_df = data.copy()
    mom3_df = mom_df.apply(lambda x:x/x.shift(3) - 1, axis=0)
    mom6_df = mom_df.apply(lambda x:x/x.shift(6) - 1, axis=0)
    mom12_df = mom_df.apply(lambda x:x/x.shift(12) - 1, axis=0)

    comb_mom_df = mom12_df+mom6_df+mom3_df

    returns_df = data.copy()
    returns_df = returns_df.apply(lambda x:x.shift(-1)/x - 1, axis=0)
    
    rank_df=comb_mom_df.rank(axis=1,ascending=False)
    
    for col in rank_df.columns:
        rank_df[col]=np.where(rank_df[col]<=assets_owned, 1, 0)
        
    portf=np.multiply(rank_df,returns_df)
    portf_rets=portf.sum(axis =1)/assets_owned

    portf_rets = portf_rets[13:]
    portf_rets = pd.DataFrame(portf_rets)
    portf_rets.columns = ['TF']

    portf_rets = portf_rets+1
    portf_rets = portf_rets.cumprod()
    portf_rets = portf_rets.astype('float64')
    
    return portf_rets

In [18]:
def bond_return(filename, sheet_no=0, header=0):
    '''10 YEAR BOND RETURN. sheet1:Original US, sheet2: Improved US, sheet3:germany, 
    sheet4:Japan, sheet5:Australia, sheet6:Norway. sheet7:Sweden'''
    
    tab = ezodf.opendoc(filename="../Data/"+filename).sheets[sheet_no]
    temp_data= pd.DataFrame({col[header].value:[x.value for x in col[header+1:]]
                         for col in tab.columns()})
    temp_data['observation_date'] = pd.to_datetime(temp_data['observation_date'], errors='coerce')
    temp_data.index = temp_data['observation_date']
    bond_ret = pd.DataFrame(temp_data['Cum Ret'])
    bond_ret = bond_ret.dropna()
    bond_ret.columns = ['10Y Bond']
    return(bond_ret)

## Data Imports and Wrangling

In [19]:
# 10 Yr Bond
path = 'International Monthly Government Bond Returns.ods'
Bond_monthly = bond_return(filename=path, sheet_no = 2)
Bond_monthly.index = Bond_monthly.index + pd.Timedelta('1 day')



# Gold

Gold_tmp = quandl.get("LBMA/GOLD", authtoken="6rNSyTM5R4g7_B7XSyQW")
Gold = pd.DataFrame(Gold_tmp["USD (AM)"])
Gold = Gold.resample('M').last()
Gold.index = Gold.index + pd.Timedelta("1 day")

# Country index
USA = import_MSCI('USA')
RUSSIA = import_MSCI('RUSSIA')
SWEDEN = import_MSCI('SWEDEN')
Emerging_Markets = import_MSCI('Emerging_Markets')
JAPAN = import_MSCI('JAPAN')

## Parameters for Trendfollowing Portfolio 

In [22]:
securities = [USA, JAPAN, Gold]
assets_owned = 1

TF_monthly = wrangling(securities, assets_owned)